In [ ]:
! pip install -U -q langchain, chromadb, langchain-core, langchain-gooogle-vertexai gradio, unstructure, google-cloud-storage, langchain-core

In [ ]:
! source .venv/bin/activate

In [1]:
# import libraries 
from langchain_google_vertexai import VertexAI
from google.cloud import storage
from langchain import LLMChain

In [2]:
# retrievals 
from langchain_community.document_loaders import GCSDirectoryLoader

In [3]:
BUCKET_NAME = "lloyds-genai24lon-2701-bucket"
PROJECT_ID = "lloyds-genai24lon-2701"

In [29]:
llm = VertexAI(model_name = 'text-bison-32k', max_output_tokens = 256, temperature = 0.1, top_p = 0.8, 
               top_k = 40, verbose = True)

In [5]:
loader = GCSDirectoryLoader(project_name = PROJECT_ID, bucket=BUCKET_NAME, prefix="sample_50/")

In [6]:
documents = loader.load()

In [30]:
CHUNK_SIZE = 1000
CHUNK_OVERLAP = 100

In [31]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=CHUNK_SIZE, chunk_overlap=CHUNK_OVERLAP)

In [32]:
docs = text_splitter.split_documents(documents)
print("splitting...")
print(f"No of Documents = {len(docs)}")

splitting...
No of Documents = 1111


In [33]:
from langchain_google_vertexai import VertexAIEmbeddings

In [34]:
# Structure ingested documents 

In [35]:
REQUESTS_PER_MINUTE = 50

In [36]:
embeddings = VertexAIEmbeddings(requests_per_minute= REQUESTS_PER_MINUTE)

Model_name will become a required arg for VertexAIEmbeddings starting from Feb-01-2024. Currently the default is set to textembedding-gecko@001


In [14]:
# Vector Store

In [37]:
from langchain_community.vectorstores import Chroma

In [38]:
documents_vector_db = Chroma.from_documents(docs, embeddings)

In [51]:
retriever = documents_vector_db.as_retriever(search_type = 'mmr', search_kwargs = {"k":10})

In [18]:
# Retrieval QA Chain 

In [52]:
from langchain.chains import RetrievalQA

In [53]:
qa = RetrievalQA.from_chain_type(llm = llm, chain_type = "stuff", retriever=retriever, return_source_documents=True)

In [54]:
result = qa({"query": "what is the context of the first page"})

In [55]:
result

{'query': 'what is the context of the first page',
 'result': ' The first page provides an overview of the approach taken by the Ombudsman in reaching their conclusions. It explains that where evidence is incomplete, inconclusive, or contradictory, they will reach their conclusions on the balance of probabilities based on the available evidence and wider surrounding circumstances. It also lists the applicable rules, regulations, and requirements that were considered in the assessment, including the Principles for Business (PRIN) and the Conduct of Business Sourcebook (COBS).',
 'source_documents': [Document(page_content="Sourcebook (COBS). And where the evidence is incomplete, inconclusive or contradictory,\n\nI reach my conclusions on the balance of probabilities  that is, what I think is more likely\n\nthan not to have happened based on the available evidence and the wider surrounding\n\ncircumstances.\n\nThe applicable rules, regulations and requirements\n\nThe below is not a compre

### Gradio 

In [56]:
def chatbot(inputtext):
    result = qa({"query": inputtext})
    return result['result'], get_public_url(result['source_documents'][0].metadata['source']), result['source_documents'][1].metadata['source']

In [57]:
def get_public_url(url): 
    components = url.split("/")
    bucket_name = components[2]
    file_name = components[3]
    client = storage.Client()
    bucket = client.bucket(bucket_name)
    blob = bucket.blob(file_name)
    return blob.public_url

In [25]:
import gradio as gr

/home/jupyter/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [47]:
examples = ['What is the context of the first page']

In [48]:
iface = gr.Interface(fn=chatbot, 
                     inputs = [gr.Textbox(label="Query")], 
                     examples=examples, title = "FOS chatbot", 
                     outputs = [gr.Textbox(label="Response"), 
                               gr.Textbox(label="URL"), gr.Textbox(label = "Cloud Storage URL")])

In [49]:
iface.launch(share=True)

Running on local URL:  http://127.0.0.1:7861
Running on public URL: https://178336a8dfb9f53155.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
